我们通过一个在控制台中旋转显示 `"|/-\"` 的动画指针来展示用线程和 asyncio 的不同

注意下面的两个代码在控制台可以显示动画，在这里不行

In [1]:
import threading
import itertools
import time
import sys

class Signal:
    go = True
  

def spin(msg, signal):
    write, flush = sys.stdout.write, sys.stdout.flush
    for char in itertools.cycle('|/-\\'):
        status = char + ' ' + msg
        write(status)
        flush()
        write('\x08' * len(status)) # \x08 是退格符
        time.sleep(.1)
        if not signal.go:
            break
    write(' ' * len(status) + '\x08' * len(status)) # 最后用空格擦除屏幕内容，把光标移回开头
    
def slow_function():
    time.sleep(3) # 3 秒动画
    return 42

def supervisor():
    signal = Signal()
    spinner = threading.Thread(target=spin,
                               args=('thinking!', signal))
    print('spinner object:', spinner)
    spinner.start()
    result = slow_function()
    signal.go = False
    spinner.join()
    return result

def main():
    result = supervisor()
    print('Answer:', result)
    
main()

spinner object: <Thread(Thread-4, initial)>
| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking          Answer: 42


In [2]:
import asyncio
import itertools
import sys

@asyncio.coroutine # 打算交给 asyncio 处理的使用此装饰器，不是强制要求，但强烈建议这么做，原因后面讲
def spin(msg): # 这里不需要上面例子中的 signal 参数
    write, flush = sys.stdout.write, sys.stdout.flush
    for char in itertools.cycle('|/-\\'):
        status = char + ' ' + msg
        write(status)
        flush()
        write('\x08' * len(status))
        try:
            yield from asyncio.sleep(.1) # 代替 time.sleep(.1)，这样休眠不会阻塞事件循环
        except asyncio.CancelledError: # 如果 spin 函数苏醒后抛出此异常，其原因是发出了取消请求，因此退出循环
            break
    write(' ' * len(status) + '\x08' * len(status))
  
@asyncio.coroutine
def slow_function(): # 现在这个函数是协程，在用休眠假装进行 I/O 操作时，使用 yield from 继续执行事件循环
    yield from asyncio.sleep(3) # 将控制权交给主循环，3 秒后休眠结束恢复此线程
    return 42

@asyncio.coroutine
def supervisor(): # 这个函数是协程，因此可以使用 yield from 驱动 slow_function() 函数
    spinner = asyncio.async(spin('thinking!')) # 排定 spin 协程的运行时间，使用一个 Task 对象包装 spin 协程，并立即返回
    print('spinner object:', spinner)
    result = yield from slow_function() # 驱动 slow_function(), 结束后获取返回值，同时事件继续执行，因为此函数 sleep 将控制权交回主循环
    spinner.cancel() # Task 对象可以取消，取消后会在协程当前暂停的 yield 抛出 asyncio.CancelledError 异常，协程可以捕获这个异常，也可以延时取消，甚至可以不取消
    return result

def main():
    loop = asyncio.get_event_loop() # 获取事件循环的引用
    result = loop.run_until_complete(supervisor()) # 驱动 supervisor，让其运行完毕，这个协程返回的是这次调用的返回值
    loop.close()
    print('Answer:', result)
    
main()

spinner object: <Task pending coro=<spin() running at <ipython-input-2-b2636da91070>:5>>
| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking          Answer: 42


使用 @asyncio.coroutine 装饰器不是强制要求，但是强烈建议这么做，因为这样能在一众普通的函数中把协程凸现出来，也有助于调试，如果还没从中产出值，协程就被垃圾回收了（意味着有操作未完成，因此可能是个缺陷），那就可以发出警告。这个装饰器不会预激协程

注意这两个函数的不同

In [2]:
# def supervisor():
#     signal = Signal()
#     spinner = threading.Thread(target=spin,
#                                args=('thinking!', signal))
#     print('spinner object:', spinner)
#     spinner.start()
#     result = slow_function()
#     signal.go = False
#     spinner.join()
#     return result

# @asyncio.coroutine
# def supervisor(): # 这个函数是协程，因此可以使用 yield from 驱动 slow_function() 函数
#     spinner = asyncio.async(spin('thinking!')) # 排定 spin 协程的运行时间，使用一个 Task 对象包装 spin 协程，并立即返回
#     print('spinner object:', spinner)
#     result = yield from slow_function() # 驱动 slow_function(), 结束后获取返回值，同时事件继续执行，因为此函数 sleep 将控制权交回主循环
#     spinner.cancel() # Task 对象可以取消，取消后会在协程当前暂停的 yield 抛出 asyncio.CancelledError 异常，协程可以捕获这个异常，也可以延时取消，甚至可以不取消
#     return result

主要区别如下：

- asyncio.Task 对象差不多与 threading.Thread 对象等效
- Task 对象用于驱动协程，Thread 对象用于调用可调用对象
- Task 对象不由自己手动实例化，而是通过把协程川贝 asyncio.async(...) 函数或 loop.create_task(...) 方法获取
- 获取的 Task 对象已经排定了运行时间（例如，由 asyncio.async 函数排定）；Thread 实例则必须调用 start 方法，明确告知让它运行
- 在线程版 supervisor 函数中，slow_function() 函数是普通的函数，直接由线程调用。在异步版 supervisor 函数中，此函数是协程，由 yield from 驱动
- 没有 API 能从外部终止线程，因为线程随时可能被中断，导致系统处于无效状态。如果想终止任务，可以使用 Task.cancel() 实例方法，在协程内部抛出 CancelledError 异常。协程可以在暂停的 yield 处捕获这个异常，处理终止请求
- supervisor 协程必须在 main 函数中由 loop.run_until_complete 方法执行

线程与协程还有一点区别要说明：如果使用线程编程，调度程序任何时候都能中断线程，必须记住保留锁，去保护程序中的重要部分，防止多步操作在执行的过程中中断，防止数据处于无效状态

协程默认会做好全方位包含，防止中断，我们必须显式产出才能让程序的余下部分运行，对于协程来说，无需保留锁，在多个线程之间同步操作，协程自身就会同步，因此在任意时刻只有一个协程运行，想交出控制权时，可以使用 yield 或 yield from 把控制权交还给调度程序。这就是能够安全地取消协程的原因：按照定义，协程只能在暂停的 yield 处取消，因此可以处理 CancelledError 异常，执行清理操作

## asyncio.Future 故意不阻塞

asyncio.Future 类和 concurrent.futures.Futer 类的接口基本一致，不过实现方式不同，不能互换。

期物只是调度执行某物的结果，在 asyncio 包中，BaseEventLoop.create_task(...) 方法来接收一个协程，排定它的运行时间，然后返回一个 asyncio.Task 实例 -- 也是 asyncio.Future 类的实例，因为 Task 是 Future 的子类，用于包装协程，这与调用 Executor.submit(...) 方法创建 concurrent.futures.Future 实例是一个道理

与 concurrent.futures.Future 类似，asyncio.Future 类也提供了 .done(), .add_done_callback(...) 和  .result()  等方法。前两个方法的用法与上一章例子一样，不过 .result() 方法差别很大

asyncio.Future 类的 .result() 方法没有参数，因此不能指定超时时间，此外，如果调用 .result() 方法时期物还没运行完毕，.result() 也不会阻塞等待结果，而是抛出 asyncio.InvalidStateError 异常

然而，获取 asyncio.Future 对象通常使用 yield from，从中产出结果

使用 yield from 处理期物，等待期物运行完毕这一步无需我们关心，而且不会阻塞事件循环，因为在 asyncio 包中，yield from 作用是把控制权交还给事件循环

注意，使用 yield from 处理期物与使用 add_done_callback 方法处理协程的作用一样：延时操作结束后，事件循环不会触发回调对象，而是设置期物的返回值，而 yield from 表达式则在暂停的协程中产生返回值，恢复执行协程

总之，因为 asyncio.Future 类的目的是与 yield from 一起使用，所以通常不需要使用以下方法

- 无需调用 my_future.add_done_callback(...)，因为可以直接把想在期物运行结束后执行的操作放到协程中的 yield from my_future 表达式后面。这是协程的一大优势：协程是可以暂停和恢复的函数
- 无需调用 my_future.result()，因为 yield from 从期物中产出的值就是结果（例如 result = yield from my_future) 

当然，有时也是用这些方法，但是一般情况下， asyncio.Future 对象是由 yield from 驱动，而不是靠调用这些方法驱动

## 从期物、任务和协程中产出

在 asyncio 包中，期物与协程关系紧密，因为可以使用 yield from 从 asyncio.Future 对象中产出结果。这意味着，如果 foo 是协程函数（调用后返回协程对象），抑或是返回 Future 或 Task 实例的普通函数，那么可以这样写: res = yield from foo() 。这是 asyncio 包的 API中很多地方可以互换协程与期物的原因之一

为了执行这些操作，必须排定协程运行时间，然后使用 asyncio.Task 对象包装协程。对协程来说，获取 Task 对象有两种主要方式

asyncio.async(coro_or_future, *, loop=None)

- 这个函数统一了协程与期物：第一个参数可以是二者中的任意一个。如果是 Future 或 Task对象，那就原封不动的返回，如果是协程，那么 async 函数会调用 loop.create_task(...) 方法创建 Task 对象，loop 关键字是可选的用于传入事件循环；如果没有传入，那么 async 函数会通过调用 asyncio.get_event_loop() 函数获取循环对象

BaseEventLoop.create_task(coro)

- 这个方法排定协程的执行时间，返回一个 asyncio.Task 对象。如果在自定义的 BaseEventLoop 子类上调用，返回的对象可能是外部库（如 Tornado）中与 Task 类兼容的某个类的实例

> BaseEventLoop.create_task(...) 只能在 Python 3.4.2 及以上版本使用，Python 3.3 或 Python 3.4 要使用 asyncio.async(...) 函数或从 PyPI 中安装新的 asyncio 版本

asyncio 包中有多个函数会自动（内部使用的是 asyncio.async 函数）把参数指定的协程包装在 asyncio.Task 对象中，例如 BaseEventLoop.run_until_complete(...) 方法

如果想在控制台或小型测试脚本中试验期物和协程，可以使用下面代码：

```
import asyncio
def run_sync(coro_or_future):
    loop = asyncio.get_event_loop()
    return loop.run_until_complete(coro_or_future)
    
a = run_sync(some_coroutine())
```

在 asyncio 文档中有个注解说道：这份文档把一些方法说成协程，即使它们其实是返回 Future 对象的普通 Python 函数，这是故意的，为的是给以后修改这些函数的实现留下余地

## 使用 asyncio 和  aiohttp 包下载

In [2]:
import asyncio
import aiohttp # 不是标准库中的，需要 pip 安装
import time

POP20_CC = ('CN IN US ID BR PK NG BD RU JP '
            'MX PH VN ET EG DE IR TR CD FR').split()

BASE_URL = 'http://flupy.org/data/flags'
DEST_DIR = 'downloads/'

def save_flag(img, filename):
    path = os.path.join(DEST_DIR, filename)
    with open(path, 'wb') as fp:
        fp.write(img)
        
def show(text):
    print(text, end=' ') # 输出末尾的换行符变成了空格
    sys.stdout.flush() #Python 中正常情况下,遇到换行才会刷新 stdout 缓冲。所以这里手动刷新缓冲
    
def main(download_many):
    t0 = time.time()
    count = download_many(POP20_CC)
    elapsed = time.time() - t0
    msg = '\n{} flags downloaded in {:.2f}s'
    print(msg.format(count, elapsed))

@asyncio.coroutine
def get_flag(cc):
    url = '{}/{cc}/{cc}.gif'.format(BASE_URL, cc = cc.lower())
    resp = yield from aiohttp.request('GET', url)
    image = yield from resp.read()
    return image

@asyncio.coroutine
def download_one(cc):
    image = yield from get_flag(cc)
    show(cc)
    save_flag(image, cc.lower() + '.gif')
    return cc

def download_many(cc_list):
    loop = asyncio.get_event_loop()
    to_do = [download_one(cc) for cc in sorted(cc_list)]
    wait_coro = asyncio.wait(to_do)
    res, _ = loop.run_until_complete(wait_coro)
    loop.close()
    return len(res)

main(download_many)

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7f4fa55e72b0>


JP 

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7f4fa454d6a0>


VN 

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7f4fa55e7e10>


...

MX 

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7f4fa4553240>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7f4fa455def0>


PH IR 

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7f4fa55f1f60>


US 
20 flags downloaded in 0.70s
